In [42]:
excel_file = 'Aaron_Cahill_Essence_Take_home_assignment (1).xlsx'

In [65]:
#importing required modules

import pandas as pd
import numpy as np
import gspread
from oauth2client.service_account import ServiceAccountCredentials
from pandas.io.json import json_normalize




In [66]:
Google_Ads_Raw_Data = pd.read_excel(excel_file, sheet_name=3)
Bing_Ads_Raw_Data = pd.read_excel(excel_file, sheet_name=4)
Media_Plan_Raw_Data = pd.read_excel(excel_file, sheet_name=5)
Mapping_Table = pd.read_excel(excel_file, sheet_name=6)


Budget_Spend_Table = pd.read_excel(excel_file, sheet_name=10)  
#Budget_Spend_Sheet = sh.get_worksheet(10) #I will eventually send the required data this tab


In [67]:
Budget_Spend_Sheet

<Worksheet 'Budget_Spend_Data' id:593478596>

In [48]:
#FIRST TABLE

import pandas as pd

Mapped_Media = Mapping_Table.merge(Media_Plan_Raw_Data, left_on = ['Corresponding Media Plan Line Id','Platform'], right_on = ['Plan Line Id', 'Platform'], how = 'inner')

#Within the Mapping Table, two separate campaigns(one from Bing, one from Google) have an identical name. While I have a column specifying the platform, no such column exists in the Raw Data tables. This makes a merge awkward. The summed value of spend per campaign (within the raw ads table) will be counted twice (as it is mentioned twice within the mapping table. Thus, I will split the Ads table into two). 

Mapped_Media_Google = Mapped_Media.loc[Mapped_Media['Platform'] == 'Google Ads']
Mapped_Media_Bing = Mapped_Media.loc[Mapped_Media['Platform'] == 'Bing Ads']


#df.loc[df['column_name'] == some_value]
#Mapping_Table
#Plan Line Id

In [49]:
##GOOGLE ADS SPEND TABLE
Summed_Spend_by_campaign_google = Google_Ads_Raw_Data.groupby(['Campaign Name'], as_index=False)['Spend'].sum()

#Summed_Spend_by_campaign.columns
#total_spend_google_ads = Summed_Spend_by_campaign_google['Spend'].sum()    #THIS IS CORRECT

In [52]:
#JOINING GOOGLE ADS TABLES

joining_tables = Summed_Spend_by_campaign_google.merge(Mapped_Media_Google, left_on = 'Campaign Name', right_on = 'Campaign')

#As Budget is grouped by Plan Line id, I will now group spend by Plan Line id. This makes a direct comparison between budget and spend possible. 
my_final_spend = joining_tables.groupby(['Plan Line Id'], as_index=False)['Spend'].sum()

my_final_spend

#joining_tables.head()
#joining_tables.shape
#d = Summed_Spend_by_campaign['Campaign Name']
#d
#q = All_Ads_Data['Campaign Name'].unique()
#len(q)
#Mapped_Media.shape

,Plan Line Id,Spend
0,123,12278.34
1,124,7417.77
2,234,21955.22
3,235,9331.26
4,345,2004.46
5,347,1106.23
6,456,3868.51
7,457,12948.29


In [53]:
the_final_budget = Mapped_Media_Google[['Plan Line Id', 'Budget']]

the_final_budget_amended = the_final_budget.groupby(['Plan Line Id'], as_index=False)['Budget'].mean()
the_final_budget_amended

,Plan Line Id,Budget
0,123,12000
1,124,7500
2,234,22000
3,235,9350
4,345,2000
5,347,1200
6,456,4000
7,457,13000


In [54]:
end_result = my_final_spend.merge(the_final_budget_amended)
end_result  

,Plan Line Id,Spend,Budget
0,123,12278.34,12000
1,124,7417.77,7500
2,234,21955.22,22000
3,235,9331.26,9350
4,345,2004.46,2000
5,347,1106.23,1200
6,456,3868.51,4000
7,457,12948.29,13000


In [55]:
#Same process for Bing Ads

##BING ADS SPEND TABLE
Summed_Spend_by_campaign_bing = Bing_Ads_Raw_Data.groupby(['Campaign Name'], as_index=False)['Spend'].sum()

#JOINING BING ADS TABLES
joining_bing_tables = Summed_Spend_by_campaign_bing.merge(Mapped_Media_Bing, left_on = 'Campaign Name', right_on = 'Campaign')

#As Budget is grouped by Plan Line id, I will now group Spend by Plan Line id. This makes a direct comparison between budget and spend possible. 
my_final_spend_bing = joining_bing_tables.groupby(['Plan Line Id'], as_index=False)['Spend'].sum()
my_final_spend_bing

#Now retrieving the budget for bing campaigns
the_final_bing_budget = Mapped_Media_Bing[['Plan Line Id', 'Budget']]

the_final_bing_budget_amended = the_final_bing_budget.groupby(['Plan Line Id'], as_index=False)['Budget'].mean()
the_final_bing_budget_amended

#Putting it all together
end_result_bing = my_final_spend_bing.merge(the_final_bing_budget_amended)
end_result_bing

,Plan Line Id,Spend,Budget
0,240,6589.093,6600
1,242,2246.093,2300
2,458,1137.469,1200
3,459,3518.700,3500


In [56]:
#Apending the two tables back together
put_together = end_result.append(end_result_bing, ignore_index=True)
put_together

,Plan Line Id,Spend,Budget
0,123,12278.340,12000
1,124,7417.770,7500
2,234,21955.220,22000
3,235,9331.260,9350
4,345,2004.460,2000
5,347,1106.230,1200
6,456,3868.510,4000
7,457,12948.290,13000
8,240,6589.093,6600
9,242,2246.093,2300


In [75]:
#Now to create the table that may serve as the View for the marketing team

Dataframe_to_send = Media_Plan_Raw_Data.merge(put_together)
Dataframe_to_send

,Media Plan,Media Plan Id,Platform,Plan Line,Plan Line Id,Start Date,End Date,Budget,Spend
0,Product 1 | France | SEM,15,Google Ads,Brand Keywords,123,2019-07-01,2019-07-31,12000,12278.340
1,Product 1 | France | SEM,15,Google Ads,Generic Keywords,124,2019-07-01,2019-07-10,7500,7417.770
2,Product 1 | UK | SEM,17,Google Ads,Brand Keywords,234,2019-07-01,2019-07-31,22000,21955.220
3,Product 1 | UK | SEM,17,Google Ads,Generic Keywords,235,2019-07-01,2019-07-15,9350,9331.260
4,Product 1 | UK | SEM,17,Bing Ads,Brand Keywords,240,2019-07-01,2019-07-31,6600,6589.093
5,Product 1 | UK | SEM,17,Bing Ads,Generic Keywords,242,2019-07-01,2019-07-15,2300,2246.093
6,Product 2 | France | SEM,23,Google Ads,Brand Keywords,345,2019-07-01,2019-07-31,2000,2004.460
7,Product 2 | France | SEM,23,Google Ads,Generic Keywords,347,2019-07-01,2019-07-31,1200,1106.230
8,Product 2 | UK | SEM,24,Google Ads,Brand Keywords,456,2019-07-01,2019-07-31,4000,3868.510
9,Product 2 | UK | SEM,24,Google Ads,Generic Keywords,457,2019-07-01,2019-07-31,13000,12948.290


In [78]:
#Need to convert the states to strings before they can be sent over to google sheets

Dataframe_to_send['Start Date'] = Dataframe_to_send['Start Date'].astype(str) 
Dataframe_to_send['End Date'] = Dataframe_to_send['Start Date'].astype(str) 
                  

#df['Date'] = df['Date'].astype(str)


In [79]:
#SORTING THE GOOGLE SHEETS PART

#Assigning variable to the link of the sheet in question

scope = ['https://docs.google.com/spreadsheets/d/1EecGKdMSqWXzTxfjEfYPCoK0ph-yAOSkWyoj_5z9an8/edit?usp=sharing']


#https://docs.google.com/spreadsheets/d/1EecGKdMSqWXzTxfjEfYPCoK0ph-yAOSkWyoj_5z9an8/edit?usp=sharing

credentials = ServiceAccountCredentials.from_json_keyfile_name('./cool-benefit-290520-30a2c738b36d (1).json', scope)
gc = gspread.authorize(credentials)
gc = gspread.service_account(filename='./cool-benefit-290520-30a2c738b36d (1).json')
sh = gc.open_by_key('1EecGKdMSqWXzTxfjEfYPCoK0ph-yAOSkWyoj_5z9an8')



In [80]:

#Budget_Spend_Table = pd.read_excel(excel_file, sheet_name=10)  

Budget_Spend_Sheet = sh.get_worksheet(10) #I will eventually send the required data this tab

In [76]:
Budget_Spend_Sheet 
Budget_Spend_Sheet.insert_row(['qqqq'], index=2)

{'spreadsheetId': '1EecGKdMSqWXzTxfjEfYPCoK0ph-yAOSkWyoj_5z9an8',
 'updatedRange': 'Budget_Spend_Data!A2',
 'updatedRows': 1,
 'updatedColumns': 1,
 'updatedCells': 1}

In [ ]:
Pivot_Data_worksheet.update([Final_Data_Set.columns.values.tolist()] + Final_Data_Set.values.tolist())

In [81]:
Budget_Spend_Sheet.update([Dataframe_to_send.columns.values.tolist()] + Dataframe_to_send.values.tolist())

{'spreadsheetId': '1EecGKdMSqWXzTxfjEfYPCoK0ph-yAOSkWyoj_5z9an8',
 'updatedRange': 'Budget_Spend_Data!A1:I13',
 'updatedRows': 13,
 'updatedColumns': 9,
 'updatedCells': 117}

In [ ]:
#The data has now been sent to the Budget_Spend_Data - to be assorted into a View